In [1]:
import sys
import numpy as np
import pandas as pd
import os
import cv2
import wandb
from datetime import datetime
from tqdm import tqdm

import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import ConcatDataset

sys.path.append('../')
from dataset.EyePACS_and_APTOS import Eye_APTOS
from dataset.messidor import MESSIDOR

# Which GPU to use
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [2]:
Eye_APTOS_data_dir_options = {
    'EyePACS': '/home/xiangjianhou/hc701-fed/preprocessed/eyepacs',
    'APTOS': '/home/xiangjianhou/hc701-fed/preprocessed/aptos',
}
MESSIDOR_data_dir_options = {
    'messidor2': '/home/xiangjianhou/hc701-fed/preprocessed/messidor2',
    'messidor_pairs' : '/home/xiangjianhou/hc701-fed/preprocessed/messidor/messidor_pairs',
    'messidor_Etienne' : '/home/xiangjianhou/hc701-fed/preprocessed/messidor/messidor_Etienne',
    'messidor_Brest-without_dilation' : '/home/xiangjianhou/hc701-fed/preprocessed/messidor/messidor_Brest-without_dilation'
}

In [3]:
Eye_APTOS_data_dir_options = {
    'EyePACS': '/home/xiangjianhou/hc701-fed/preprocessed/eyepacs',
    'APTOS': '/home/xiangjianhou/hc701-fed/preprocessed/aptos',
}


MESSIDOR_data_dir_options = {
    'messidor2': '/home/xiangjianhou/hc701-fed/preprocessed/messidor2',
    'messidor_pairs' : '/home/xiangjianhou/hc701-fed/preprocessed/messidor/messidor_pairs',
    'messidor_Etienne' : '/home/xiangjianhou/hc701-fed/preprocessed/messidor/messidor_Etienne',
    'messidor_Brest-without_dilation' : '/home/xiangjianhou/hc701-fed/preprocessed/messidor/messidor_Brest-without_dilation'
}


APTOS_train = Eye_APTOS(data_dir=Eye_APTOS_data_dir_options['APTOS'], train=True, transform=None)
EyePACS_train = Eye_APTOS(data_dir=Eye_APTOS_data_dir_options['EyePACS'], train=True, transform=None)
MESSIDOR_2_train = MESSIDOR(data_dir=MESSIDOR_data_dir_options['messidor2'], train=True, transform=None)
MESSIDOR_pairs_train = MESSIDOR(data_dir=MESSIDOR_data_dir_options['messidor_pairs'], train=True, transform=None)
MESSIDOR_Etienne_train = MESSIDOR(data_dir=MESSIDOR_data_dir_options['messidor_Etienne'], train=True, transform=None)
MESSIDOR_Brest_train = MESSIDOR(data_dir=MESSIDOR_data_dir_options['messidor_Brest-without_dilation'], train=True, transform=None)

Centerlized_train = ConcatDataset([APTOS_train, EyePACS_train, MESSIDOR_2_train, MESSIDOR_pairs_train, MESSIDOR_Etienne_train,MESSIDOR_Brest_train])
MESSIDOR_Centerlized_train = ConcatDataset([MESSIDOR_pairs_train, MESSIDOR_Etienne_train,MESSIDOR_Brest_train])

APTOS_test = Eye_APTOS(data_dir=Eye_APTOS_data_dir_options['APTOS'], train=False, transform=None)
EyePACS_test = Eye_APTOS(data_dir=Eye_APTOS_data_dir_options['EyePACS'], train=False, transform=None)
MESSIDOR_2_test = MESSIDOR(data_dir=MESSIDOR_data_dir_options['messidor2'], train=False, transform=None)
MESSIDOR_pairs_test = MESSIDOR(data_dir=MESSIDOR_data_dir_options['messidor_pairs'], train=False, transform=None)
MESSIDOR_Etienne_test = MESSIDOR(data_dir=MESSIDOR_data_dir_options['messidor_Etienne'], train=False, transform=None)
MESSIDOR_Brest_test = MESSIDOR(data_dir=MESSIDOR_data_dir_options['messidor_Brest-without_dilation'], train=False, transform=None)

Centerlized_test = ConcatDataset([APTOS_test, EyePACS_test, MESSIDOR_2_test, MESSIDOR_pairs_test, MESSIDOR_Etienne_test,MESSIDOR_Brest_test])
MESSIDOR_Centerlized_test = ConcatDataset([MESSIDOR_pairs_test, MESSIDOR_Etienne_test,MESSIDOR_Brest_test])

In [4]:
train_dataset = DataLoader(Centerlized_train, batch_size=32, shuffle=True)
MESSIDOR_train_dataset = DataLoader(MESSIDOR_Centerlized_train, batch_size=32, shuffle=True)

In [6]:
from model.baseline import Baseline
model_demo = Baseline(backbone='vit_tiny_patch16_224',num_classes=5,pretrained=True)
LOSS = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_demo.parameters(), lr=0.001)
model_save_path = '/home/xiangjianhou/hc701-fed/checkpoint/MESSIDOR_3_hosptial_4class'

In [7]:
model_demo.train()
model_demo.to(device)
for epoch in range(1):
    model_demo.train()
    model_demo.to(device)
    train_loss = 0
    train_acc = 0
    for i, (x, y) in enumerate(tqdm(train_dataset)):
        x = x.to(device,torch.float32)
        y = y.to(device,torch.long)
        pred = model_demo(x)
        loss = LOSS(pred, y)
        train_loss += loss.item()
        train_acc += (pred.argmax(dim=1) == y).sum().item()
    train_loss /= len(train_dataset)
    train_acc /= len(train_dataset.dataset)
    print('epoch: {}, train loss: {:.4f}, train acc: {:.4f}'.format(epoch, train_loss, train_acc))

 42%|████▏     | 534/1282 [00:35<00:44, 16.91it/s]